# Cài đặt một số thư viện 

In [34]:
# %pip install underthesea 
# %pip install pyvi


In [35]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import unicodedata
import re
import regex
import pyvi


# Đọc dữ liệu từ bộ dataset 

In [36]:
df = pd.read_csv(r"D:\Spring2025\DAP391m\Fake-shopee-comment--main\shopee_data\Generated_Sentences_1.csv")

xem 5 dòng đầu tiên

In [37]:
df.head()

,category,text,label
0,Thời Trang Nữ,Siuu đẹp:))\n,CG
1,Thời Trang Nam,Dùng được so với giá tiền,CG
2,Túi Ví Nữ,Xinh lắm mn nên mua nha,CG
3,Giày Dép Nam,"Đẹp, rẻ nên mua nha",CG
4,Balo & Túi Ví Nam,Giao hàg nhah k,CG


xem một số thông tin cơ bản

In [38]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68083 entries, 0 to 68082
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   category  68083 non-null  object
 1   text      68083 non-null  object
 2   label     68083 non-null  object
dtypes: object(3)
memory usage: 1.6+ MB


In [39]:
# xem số lượng trong category
print(f"Số lượt đánh giá trong các loại sản phẩm\n {df['category'].value_counts()}")

Số lượt đánh giá trong các loại sản phẩm
 category
Thời Trang Nữ        8766
Giày Dép Nữ          8696
Túi Ví Nữ            8575
Thời Trang Trẻ Em    8563
Thời Trang Nam       8424
Giày Dép Nam         8372
Đồng Hồ              8352
Balo & Túi Ví Nam    8335
Name: count, dtype: int64


# Tiền xử lý dữ liệu

**Các bước thực hiện tiền xử lý bộ dữ liệu theo trình tự như sau**
1. Xóa NaN trong comment
2. Normalize
3. Kiểm tra và loại bỏ các comment có chứa URL
4. Loại bỏ các ký tự đặt biệt
5. Loại bỏ các bình luận giống nhau
6. Thay thế các từ viết tắt


In [40]:
'''Pattern dùng để kiểm tra text có chứa URL hay không'''
URL = r"(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|\([^\s()<>]+\))*\))+(?:\(([^\s()<>]+|\([^\s()<>]+\))*\)|[^\s`!()\[\]{};:'\".,<>?]))"

'''Các ký tự UTF-8 in hoa (chỉ giữ lại các ký tự hợp lệ tiếng Việt)'''
UTF8_UPPER = r"[^A-ZÁÀẢÃẠÂẤẦẨẪẬĂẮẰẲẴẶÉÈẺẼẸÊẾỀỂỄỆÍÌỈĨỊÓÒỎÕỌÔỐỒỔỖỘƠỚỜỞỠỢÚÙỦŨỤƯỨỪỬỮỰÝỲỶỸỴĐ]"

'''Các ký tự UTF-8 in thường (chỉ giữ lại các ký tự hợp lệ tiếng Việt và khoảng trắng)'''
UTF8_LOWER = r"[^a-záàảãạâấầẩẫậăắằẳẵặéèẻẽẹêếềểễệíìỉĩịóòỏõọôốồổỗộơớờởỡợúùủũụưứừửữựýỳỷỹỵđ ]"

 # 1. Xóa NaN trong cột comment

In [41]:
df.dropna(subset=['text'],inplace=True)

# 2. Normalize

In [42]:
df['processed_comment'] = df['text'].astype(str).apply(lambda cmt : unicodedata.normalize('NFC', cmt.lower()))
df.sample(5)

,category,text,label,processed_comment
36603,Giày Dép Nam,Màu sắc:trắng kem\nĐúng với mô tả:đúng\n\nÁo đ...,CG,màu sắc:trắng kem\nđúng với mô tả:đúng\n\náo đ...
3041,Thời Trang Trẻ Em,Đúng với mô tả:cungx dc,OR,đúng với mô tả:cungx dc
53202,Đồng Hồ,sản phẩm nhận đc giống như ảnh và giao hàng nh...,CG,sản phẩm nhận đc giống như ảnh và giao hàng nh...
60510,Giày Dép Nữ,Quá xinh yêu luôn đó keo 😍 mà mang lâu cũng hơ...,OR,quá xinh yêu luôn đó keo 😍 mà mang lâu cũng hơ...
12308,Giày Dép Nữ,Giày về vừa in chân luôn ạ.Chất liệu:vịn\nMàu ...,CG,giày về vừa in chân luôn ạ.chất liệu:vịn\nmàu ...


# 3. Kiểm tra URL

In [43]:
df['contain_url'] = df['processed_comment'].apply(lambda cmt: int(bool(re.compile(URL).search(cmt))))
df[df['contain_url'] == 1].head()

,category,text,label,processed_comment,contain_url
2244,Đồng Hồ,https://vunhapho.com,OR,https://vunhapho.com,1
7568,Balo & Túi Ví Nam,Nhận hàng nhanh \nHttps://phuquyauto.com,OR,nhận hàng nhanh \nhttps://phuquyauto.com,1
19896,Thời Trang Trẻ Em,Hinh ảnh video mang tchat.didong.vn/iphone/iph...,CG,hinh ảnh video mang tchat.didong.vn/iphone/iph...,1
29563,Balo & Túi Ví Nam,Đúng với mô tả:ttfffgh\n\nHttp www.didongthocs...,CG,đúng với mô tả:ttfffgh\n\nhttp www.didongthocs...,1
33239,Balo & Túi Ví Nam,Tạm ổn so với giá\nAe mua giày ib ủng hộ nhé \...,OR,tạm ổn so với giá\nae mua giày ib ủng hộ nhé \...,1


Giữ lại các bình luận không chứ URL

In [44]:
df = df[df['contain_url'] == 0]
df.head()

,category,text,label,processed_comment,contain_url
0,Thời Trang Nữ,Siuu đẹp:))\n,CG,siuu đẹp:))\n,0
1,Thời Trang Nam,Dùng được so với giá tiền,CG,dùng được so với giá tiền,0
2,Túi Ví Nữ,Xinh lắm mn nên mua nha,CG,xinh lắm mn nên mua nha,0
3,Giày Dép Nam,"Đẹp, rẻ nên mua nha",CG,"đẹp, rẻ nên mua nha",0
4,Balo & Túi Ví Nam,Giao hàg nhah k,CG,giao hàg nhah k,0


In [45]:
# xóa cột contain_url
df = df.drop(columns=['contain_url']).reset_index(drop=True)
df.shape

(67845, 4)

In [46]:
df.head()

,category,text,label,processed_comment
0,Thời Trang Nữ,Siuu đẹp:))\n,CG,siuu đẹp:))\n
1,Thời Trang Nam,Dùng được so với giá tiền,CG,dùng được so với giá tiền
2,Túi Ví Nữ,Xinh lắm mn nên mua nha,CG,xinh lắm mn nên mua nha
3,Giày Dép Nam,"Đẹp, rẻ nên mua nha",CG,"đẹp, rẻ nên mua nha"
4,Balo & Túi Ví Nam,Giao hàg nhah k,CG,giao hàg nhah k


# 4. Loại bỏ các ký tự đặt biệt

In [47]:
df['processed_comment'] = df['processed_comment'].apply(lambda cmt: re.compile(r'\s+').sub(' ', re.compile(UTF8_LOWER).sub(' ', cmt)))
df.sample(5)

,category,text,label,processed_comment
17321,Balo & Túi Ví Nam,Đúng với mô tả:đúng vs mô tả\nChất liệu:vải\n\...,CG,đúng với mô tả đúng vs mô tả chất liệu vải sản...
39326,Thời Trang Nữ,"Hình ảnh và video chỉ mang tính chất nhận xu, ...",CG,hình ảnh và video chỉ mang tính chất nhận xu g...
23554,Thời Trang Trẻ Em,"Mua lần thứ n , săn đc mã giảm giá trên live n...",OR,mua lần thứ n săn đc mã giảm giá trên live nên...
24593,Balo & Túi Ví Nam,Túi đẹp đựng vừa in hình ảnh và video. Giao hà...,CG,túi đẹp đựng vừa in hình ảnh và video giao hàn...
22808,Giày Dép Nam,"Dép rất đẹp luôn ạ, săn sale có 12k cực kỳ rẻ....",OR,dép rất đẹp luôn ạ săn sale có k cực kỳ rẻ sho...


# 5. Loại bỏ các ký tự bị lặp trong một chữ

In [48]:
df['processed_comment'] = df['processed_comment'].apply(lambda cmt: re.compile(r'(.)\1+').sub(r'\1', cmt))
df.sample(5)

,category,text,label,processed_comment
43923,Giày Dép Nữ,Màu sắc:màu sắcccc màuuuuu kemm\nĐúng với mô t...,OR,màu sắc màu sắc màu kem đúng với mô tả đúng vớ...
18918,Đồng Hồ,Chất lượng: tốt\nTương tự như ad: vâng\n\nĐồng...,OR,chất lượng tốt tương tự như ad vâng đồng hồ đơ...
889,Giày Dép Nữ,Đúng với mô tả:đúng\n,CG,đúng với mô tả đúng
40488,Balo & Túi Ví Nam,Màu sắc:đe\nChất liệu:tốt\n\nHình ảnh chỉ mang...,OR,màu sắc đe chất liệu tốt hình ảnh chỉ mang tín...
53015,Túi Ví Nữ,Chất lượng: tốt\nPhù hợp với hình ảnh: có\n\nN...,OR,chất lượng tốt phù hợp với hình ảnh có người b...


# 6. Thay thế các từ viết tắt 

1. Xây dựng từ điển cho các từ viết tắt


In [49]:
file = open('teencode.txt', 'r', encoding="utf8")
teen_lst = file.read().split('\n')
teen_dict = {}
for line in teen_lst:
    key, value = line.split('\t')
    teen_dict[key] = str(value)
file.close()

# test
df['processed_comment'] = df['processed_comment'].apply(lambda cmt : unicodedata.normalize('NFC', cmt.lower())) # chuẩn hóa unicode lần nữa
teen_dict['ch']

'chưa'

2. Thay thế các từ viết tắt trong df

In [66]:
def replaceTeencode(cmt, teen_dict):
    words = cmt.split()
    for word in words:
        if word in teen_dict:
            cmt = cmt.replace(word, teen_dict[word])
    return cmt

df['processed_comment'] = df['processed_comment'].apply(lambda cmt: replaceTeencode(cmt, teen_dict))
df.head()

,category,text,label,processed_comment
0,Thời Trang Nữ,Siuu đẹp:))\n,CG,siu đẹp
1,Thời Trang Nam,Dùng được so với giá tiền,CG,dùng được so với giá tiền
2,Túi Ví Nữ,Xinh lắm mn nên mua nha,CG,xinh lắm mọi người nên mua nha
3,Giày Dép Nam,"Đẹp, rẻ nên mua nha",CG,đẹp rẻ nên mua nha
4,Balo & Túi Ví Nam,Giao hàg nhah k,CG,giao hàg nhah không


In [72]:
df = df.drop(columns = 'text')
df.head()

,category,label,processed_comment
0,Thời Trang Nữ,CG,siu đẹp
1,Thời Trang Nam,CG,dùng được so với giá tiền
2,Túi Ví Nữ,CG,xinh lắm mọi người nên mua nha
3,Giày Dép Nam,CG,đẹp rẻ nên mua nha
4,Balo & Túi Ví Nam,CG,giao hàg nhah không


In [73]:
df.columns = ['category', 'label', 'text']
df.head()

,category,label,text
0,Thời Trang Nữ,CG,siu đẹp
1,Thời Trang Nam,CG,dùng được so với giá tiền
2,Túi Ví Nữ,CG,xinh lắm mọi người nên mua nha
3,Giày Dép Nam,CG,đẹp rẻ nên mua nha
4,Balo & Túi Ví Nam,CG,giao hàg nhah không


In [74]:
df.to_csv("cleaned_data.csv",encoding='utf-8')